In [1]:
import json
import requests
import re
from IPython import display
from datetime import datetime
import pandas as pd
import time

In [2]:
FIREBASE_PRIVATE_KEY = r'C:\Users\johnp\Documents\ESportsAnalytics\esportsanalytics-3b04cf0c1d47.json'
BASE_URL = 'https://api.opendota.com/api/'
TEAMS = 'teams'
HEROES = 'heroes'
PROPLAYERS = 'proPlayers'
PROMATCHES = 'proMatches'
MATCHES = 'matches'
PLAYERS = 'players'
HEROES_STATS = 'heroStats'

In [77]:
# read team OG matches file
with open(r'C:\Users\johnp\Documents\ESportsAnalytics\team_og_matches.json', 'r', encoding="utf-8") as input_file:

    data = input_file.read()
    team_og_matches_json = json.loads(data)

In [4]:
def match_outcome(radiant_win, radiant):
    if ((radiant_win == False and radiant == False) or (radiant_win == True and radiant == True)):
        return True
    
    elif ((radiant_win == True and radiant == False) or (radiant_win == False and radiant == True)):
        return False

In [78]:
error_list = []

for index, item in enumerate(team_og_matches_json):

    predict_players_mmr_total = 0
    predict_players_mmr_count = 0
    for x, i in enumerate(item['predict_players']):

        try:
            predict_players_mmr_total += i['player_stats']['mmr_estimate']['estimate']
            predict_players_mmr_count += 1

        except:

            error_list.append(index)
            
    # get the average MMR of the team
    team_og_matches_json[index]['predict_players_mmr_average'] = predict_players_mmr_total / predict_players_mmr_count

    opposing_players_mmr_total = 0
    opposing_players_mmr_count = 0
    for y, j in enumerate(item['opposing_players']):

        try:
            opposing_players_mmr_total += j['player_stats']['mmr_estimate']['estimate']
            opposing_players_mmr_count += 1

        except:

            error_list.append(index)
            
    team_og_matches_json[index]['opposing_players_mmr_average'] = opposing_players_mmr_total / opposing_players_mmr_count
    
    
    predict_team_total_matches = item['predict_team_stat'][0]['predict_team_wins_total'] + item['predict_team_stat'][0]['predict_team_losses_total']
    team_og_matches_json[index]['predict_team_total_matches'] = predict_team_total_matches
    
    predict_team_win_pct = round(item['predict_team_stat'][0]['predict_team_wins_total'] / predict_team_total_matches * 100, 2)
    team_og_matches_json[index]['predict_team_win_pct'] = predict_team_win_pct
    
    opposing_team_total_matches = item['opposing_team_stat'][0]['opposing_team_wins_total'] + item['opposing_team_stat'][0]['opposing_team_losses_total']
    team_og_matches_json[index]['opposing_team_total_matches'] = opposing_team_total_matches 
    
    opposing_team_win_pct =  round(item['opposing_team_stat'][0]['opposing_team_wins_total'] / opposing_team_total_matches * 100, 2)
    team_og_matches_json[index]['opposing_team_win_pct'] = opposing_team_win_pct

In [48]:
for i in team_og_matches_json[0]:
    print(i)

match_id
radiant_win
radiant
duration
start_time
leagueid
league_name
cluster
opposing_team_id
opposing_team_name
opposing_team_logo
predict_players
opposing_players
opposing_team_stat
predict_team_stat
predict_players_mmr_average
opposing_players_mmr_average
predict_team_total_matches
predict_team_win_pct
opposing_team_total_matches
opposing_team_win_pct


In [87]:
# Convert the team matches json to a dataframe
def team_matches_df(team_matches_json, predict_team_name, predict_team_id):
    
    all_matches_df = pd.DataFrame(data = [(datetime.fromtimestamp(item['start_time']).strftime('%Y-%m-%d %H:%M:%S'),
                                                       predict_team_name,
                                                       predict_team_id,
                                                       item['predict_team_total_matches'],
                                                       item['predict_team_win_pct'],
                                                       item['predict_team_stat'][0]['predict_team_rating'],
                                                       item['predict_players_mmr_average'],
                                                       item['opposing_team_name'],
                                                       item['opposing_team_id'],
                                                       item['opposing_team_total_matches'],
                                                       item['opposing_team_win_pct'],
                                                       item['opposing_team_stat'][0]['opposing_team_rating'],
                                                       item['opposing_players_mmr_average'],
                                                       item['league_name'],
                                                       item['leagueid'],
                                                       item['cluster'], 
                                                       item['radiant_win'],
                                                       item['radiant'],
                                                       item['duration'], 
                                                       item['match_id'],
                                                       match_outcome(item['radiant_win'], item['radiant'])) for index, item in enumerate(team_matches_json)], 
                      columns = ['start_date', 
                                 'predict_team_name', 
                                 'predict_team_id', 
                                 'predict_team_total_matches', 
                                 'predict_team_win_pct',
                                 'predict_team_rating',   
                                 'predict_players_mmr_average', 
                                 'opposing_team_name', 
                                 'opposing_team_id', 
                                 'opposing_team_total_matches',
                                 'opposing_team_win_pct', 
                                 'opposing_team_rating',
                                 'opposing_players_mmr_average', 
                                 'league_name', 
                                 'leagueid', 
                                 'cluster', 
                                 'radiant_win', 
                                 'radiant', 
                                 'duration', 
                                 'match_id', 
                                 'match_outcome'])
    
    return all_matches_df


In [88]:
team_og_matches_json[1334]['predict_team_stat']

[{'predict_team_wins_total': 817,
  'predict_team_losses_total': 508,
  'predict_team_rating': 1554.93,
  'predict_team_name': 'OG',
  'predict_team_id': 2586976}]

In [89]:
og_team_matches_df = team_matches_df(team_og_matches_json, 'OG', 2586976)

In [90]:
og_team_matches_df.shape

(1335, 21)

In [22]:
def combine_team_matches_data(team_matches_json, predict_team_id):  
    matches_total = len(team_matches_json)
    count = 1
    
    # 1 x API CALL
    predict_team_stat = get_team_stat(predict_team_id)
    
    for index, value in enumerate(team_matches_json):
        
         # Check if we already maxed out our requests for that minute (only 60 requests per minute on the free tier)
        if (count%5 == 0):
            for i in range(60, -1, -1):
                print('Exceeded max API calls, please wait:', i, 'seconds                                                ', end = '\r')
                time.sleep(1)
                
            count = 1

        else:

            match_info = team_matches_json[index]
            
            # 1 x API CALL 
            match_info_detailed = get_match_info(match_info['match_id'])

            # 5 x API CALL (5 players)
            predict_players = [{'account_id' : player['account_id'], 
                                'name' : player['name'],
                                'player_stats' : get_player_info(player['account_id'])
                               } for player in match_info_detailed['players'] if player['isRadiant'] == match_info['radiant']]

            # 5 x API CALL (5 players)
            opposing_players = [{'account_id' : player['account_id'], 
                                 'name' : player['name'],
                                 'player_stats' : get_player_info(player['account_id'])
                                } for player in match_info_detailed['players'] if player['isRadiant'] != match_info['radiant']]

            team_matches_json[index]['predict_players'] = predict_players
            team_matches_json[index]['opposing_players'] = opposing_players

            # 1 x API CALL
            opposing_team_stat = get_team_stat(match_info['opposing_team_id'])
            
            opposing_team = [{'opposing_team_wins_total' : opposing_team_stat['wins'], 
                              'opposing_team_losses_total' : opposing_team_stat['losses'], 
                              'opposing_team_rating' : opposing_team_stat['rating']}]
            
            predict_team = [{'predict_team_wins_total' : predict_team_stat['wins'], 
                              'predict_team_losses_total' : predict_team_stat['losses'], 
                              'predict_team_rating' : predict_team_stat['rating'],
                              'predict_team_name' : predict_team_stat['name'], 
                              'predict_team_id' : predict_team_stat['team_id']}]
            
            team_matches_json[index]['opposing_team_stat'] = opposing_team
            team_matches_json[index]['predict_team_stat'] = predict_team

            print('match number:', index, 'downloaded', matches_total - index, 'matches left to go', round((index / matches_total) * 100, 2),'% complete', end = '\r')
            
            count += 1
         
    return team_matches_json

In [8]:
def get_match_info(match_id):
    response = requests.get(BASE_URL + MATCHES + '/' + str(match_id))

    try:
        match_info_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return match_info_json
    
    return 0

In [9]:
def get_player_info(account_id):
    response = requests.get(BASE_URL + PLAYERS + '/' + str(account_id))

    try:
        player_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return player_data_json
    
    return 0

In [10]:
def get_team_stat(team_id):
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id))

    try:
        team_stat_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_stat_json
    
    return 0

with open('team_og_matches.json', 'w', encoding='utf-8') as f:
    json.dump(team_og_matches_json, f, ensure_ascii=False, indent=4)

In [92]:
og_team_matches_df.columns

Index(['start_date', 'predict_team_name', 'predict_team_id',
       'predict_team_total_matches', 'predict_team_win_pct',
       'predict_team_rating', 'predict_players_mmr_average',
       'opposing_team_name', 'opposing_team_id', 'opposing_team_total_matches',
       'opposing_team_win_pct', 'opposing_team_rating',
       'opposing_players_mmr_average', 'league_name', 'leagueid', 'cluster',
       'radiant_win', 'radiant', 'duration', 'match_id', 'match_outcome'],
      dtype='object')

In [93]:
og_team_matches_df.head()

,start_date,predict_team_name,predict_team_id,predict_team_total_matches,predict_team_win_pct,predict_team_rating,predict_players_mmr_average,opposing_team_name,opposing_team_id,opposing_team_total_matches,...,opposing_team_rating,opposing_players_mmr_average,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome
0,2022-08-28 20:37:20,OG,2586976,1325,61.66,1554.93,6058.8,Team Aster,6209166,903,...,1493.25,7204.6,ESL One Malaysia 2022 powered by Intel,14388,155,True,True,2455,6728572058,True
1,2022-08-28 19:19:19,OG,2586976,1325,61.66,1554.93,6058.8,Team Aster,6209166,903,...,1493.25,7204.6,ESL One Malaysia 2022 powered by Intel,14388,155,False,False,2996,6728463079,True
2,2022-08-28 18:03:26,OG,2586976,1325,61.66,1554.93,6058.8,Team Aster,6209166,903,...,1493.25,7204.6,ESL One Malaysia 2022 powered by Intel,14388,151,False,False,2776,6728333090,True
3,2022-08-28 15:18:42,OG,2586976,1325,61.66,1554.93,6058.8,Team Secret,1838315,1489,...,1405.11,6524.6,ESL One Malaysia 2022 powered by Intel,14388,151,False,False,1628,6728136960,True
4,2022-08-28 14:04:20,OG,2586976,1325,61.66,1554.93,6058.8,Team Secret,1838315,1489,...,1405.11,6524.6,ESL One Malaysia 2022 powered by Intel,14388,156,False,False,2547,6728055442,True
